# Chains

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# LCEL：Chainの基本的な使い方

In [2]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model_name = "gpt-3.5-turbo-instruct"

template = "{subject}を勉強する方法をステップ・バイ・ステップで教えてください"

prompt = PromptTemplate.from_template(template)
model = OpenAI(model=model_name)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [3]:
chain.invoke({"subject": "英語"})

'\n\nステップ1：目標を設定する\nまずは、英語を勉強する目的や目標を明確にしましょう。自分が何を学びたいのか、どの程度のレベルを目指すのかを決めることで、勉強の方向性やモチベーションが高まります。\n\nステップ2：学習方法を決める\n次に、自分に合った学習方法を選びましょう。教室やオンラインのコース、自宅での自主学習など、様々な方法があります。自分に合った方法を選ぶことで、より効率的に学習することができます。\n\nステップ3：学習スケジュールを立てる\n学習する時間や頻'

# Chat Model

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model_name = "gpt-3.5-turbo-0125"

template = "あなたは{input_language}から{output_language}に翻訳する優秀な翻訳家です。"
human_template = "{text}"

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

model = ChatOpenAI(model=model_name)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [5]:
chain.invoke({
  "input_language": "英語",
  "output_language": "日本語",
  "text": "I love programming."})

'プログラミングが大好きです。'

# Conversation Chain：会話を続ける方法

In [6]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model_name = "gpt-3.5-turbo-0125"

model = ChatOpenAI(
    model_name=model_name,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=1.2,
)

# memoryの使用をLCELで記述する方法が不明なためConversationChainを使う
conversation = ConversationChain(llm=model, memory=ConversationBufferMemory())

In [7]:
while True:
    # ユーザーからの質問を受付
    message = input("メッセージを入力:")

    # 質問が入力されなければ終了
    if message.strip() == "":
        break
    display(f"質問:{message}")

    # 言語モデルに質問
    conversation.invoke(input=message)

print("\n---ご利用ありがとうございました！---")

'質問:こんにちは！'

 Hello! こんにちは！How are you today?

'質問:仙台でオススメのランチを教えて！'

仙台でオススメのランチスポットをいくつか教えますね！まずは定禅寺通りにある「仙花」というお店が人気です。ボリューム満点の定食やランチプレートがおすすめですよ！それから、国分町で人気の「牛たん炭焼 利久」もおいしいです。牛たんを堪能できるお店です。他にもたくさんおすすめのランチスポットがありますが、どんなメニューをお探しですか？

'質問:ラーメン'

仙台でおすすめのラーメン店の一つは、駅前のラーメン横丁にある「一幻」です。こちらは濃厚でコク深い味わいのラーメンが人気ですよ。他にも、「まるすけラーメン」や「麺鮮」なども評判がいいお店です。どのお店も味に自信があるので、ぜひ試してみてくださいね！他にも条件があれば教えてください。
---ご利用ありがとうございました！---


# RAGアーキテクチャ（LCEL)

In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# テキストファイルを読込
loader = DirectoryLoader("./data/")
documents = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

In [9]:
from langchain_openai import ChatOpenAI

model_name = "gpt-3.5-turbo-0125"

# モデルの作成
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=300,
    temperature=1.2)

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# プロンプトの作成
prompt = ChatPromptTemplate.from_template("""提供されたコンテキストのみに基づいて次の質問に答えてください:

<コンテキスト>
{context}
</コンテキスト>

Question: {input}""")

# チェーンの作成
chain = (
  {"context": retriever, "input": RunnablePassthrough()}
  | prompt 
  | model 
  | StrOutputParser()
)

In [11]:
# 質問
chain.invoke("キノコストレージについて説明して")

'キノコストレージは、容量の大きなデータストレージ装置を指す架空のIT用語であり、その形状がキノコに似ていることから名付けられています。この装置は非常に小さなサイズでありながら、膨大なデータを保持することができるとされています。また、キノコストレージは高い耐久性を持ち、データの保護と安全性を確保すると説明されています。'

# 対話型RetrievalChain

In [12]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.document_loaders import PyPDFLoader

# 複数のローダーから読込
loaders = [
    WikipediaLoader(query="SDGs", load_max_docs=3, lang="ja"),
    PyPDFLoader("./data2/20210615_resources_data_guideline_01.pdf")]

documents = []

for loader in loaders:
    documents.extend(loader.load())

documents

[Document(page_content='持続可能な開発目標（じぞくかのうなかいはつもくひょう、英語:  Sustainable Development Goals、略称: SDGs（エスディージーズ））は、2015年9月25日に国連総会で採択された、持続可能な開発のための17の国際目標である。その下に、169の達成基準と232の指標が決められている。\n\n\n== 概要 ==\n2000年9月、アメリカ・ニューヨークでの『国連ミレニアム・サミット』にて採択された国連ミレニアム宣言を基に、「ミレニアム開発目標」（MDGs）が成立した。だがMDGsは2015年までの期限付きであり、2011年、南米コロンビアのフアン・マヌエル・サントス政権にて、期限が近づいても未達成の目標があったことや、それと同時に持続可能な開発をどう進めるかについての議論が提案された。発案はコロンビア外務省経済・社会・環境の女性局長であるパウラ・カバジェーロによるものである。女性外務大臣のマリーア・アンジェラ・オルギンへの稟議を通じて採択された。\n案はグアテマラ等、中南米のいくつかの開発途上国の支援を得たのち、2012年6月にブラジル・リオデジャネイロにて開催された国際サミット『国連持続可能な開発会議』（リオ+20）にて、「2015年以降の次世代MDGsの目標と、持続可能な開発の議論を統一」させることをコロンビア・グアテマラ両国が推奨した。サミットでいかに成果を生み出すかに迷走していたブラジルは、両国の案に乗り、開催されたリオ+20では、その制定への決議を主要な成果として終了した。\n2015年9月25日、国連総会にて、持続可能な開発のために必要不可欠な向こう15年間の新たな行動計画『我々の世界を変革する：持続可能な開発のための2030アジェンダ』（Transforming our world: the 2030 Agenda for Sustainable Development、または単に「2030 Agenda」とも）が採択され、2030年までに達成するべき持続可能な開発目標（SDGs）として、17の世界的目標と169の達成基準が示された。\n\n\n== 17の目標 ==\n\nSDGsは、以下の17の目標から構成されている。その他、169項目など全文はウィキソースの「持続可能な開

In [13]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 検索（Retriever）の取得
retriever = db.as_retriever()

In [15]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

model_name = "gpt-3.5-turbo-0125"

# モデルの作成
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=300,
    temperature=1.2)

# Memoryの作成
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 会話型RetrievalQAチェーンの作成
chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    max_tokens_limit=4096,
    memory=memory)

In [16]:
# ユーザーからの質問
result = chain.invoke({"question": "SDGsとは何ですか？"})

# 言語モデルからの回答を表示
result["answer"]

'SDGs（持続可能な開発目標）は、2015年に国連で採択された17の国際目標で、持続可能な開発のための行動計画です。それらは2030年までに達成されることを目指しており、さまざまな分野にわたる持続可能な開発を促進することを目的としています。'

In [17]:
# ユーザーからの質問
result = chain.invoke({"question": "1つ目の目標について説明してください。出典も示してください"})

# 言語モデルからの回答を表示
result["answer"]

'SDGs（持続可能な開発目標）の最初の目標は、「貧困を終わらせる」というものです。この目標では、貧困者や弱者の支援、社会的な包摂、機会均等の推進が重要視されています。また、全ての国々において経済成長と繁栄を確保しつつ、貧困層や社会的に脆弱な人々が取り残されることのないようにすることが目指されています。\n\n出典：持続可能な開発のための2030アジェンダ（ウィキソース）'

In [18]:
# ユーザーからの質問
result = chain.invoke({"question": "オープンデータの定義とは？"})

# 言語モデルからの回答を表示
result["answer"]

'オープンデータの定義は、お答えの中にある「2. オープンデータとは言えないものの、データ提供システムの維持管理に要するコストを限定された利用者からの料金徴収でまかなうケースもある」という一部の一文によると、「データ提供システムの維持管理に要するコストを限定された利用者からの料金徴収でまかなう場合は、オープンデータとは言えないが、その取扱いに準じ、二次利用可能なルールを適用することが望ましい」とされています。'

In [19]:
# 保存されているメッセージを取得
buffer = memory.load_memory_variables({})
print(buffer)

{'chat_history': [HumanMessage(content='SDGsとは何ですか？'), AIMessage(content='SDGs（持続可能な開発目標）は、2015年に国連で採択された17の国際目標で、持続可能な開発のための行動計画です。それらは2030年までに達成されることを目指しており、さまざまな分野にわたる持続可能な開発を促進することを目的としています。'), HumanMessage(content='1つ目の目標について説明してください。出典も示してください'), AIMessage(content='SDGs（持続可能な開発目標）の最初の目標は、「貧困を終わらせる」というものです。この目標では、貧困者や弱者の支援、社会的な包摂、機会均等の推進が重要視されています。また、全ての国々において経済成長と繁栄を確保しつつ、貧困層や社会的に脆弱な人々が取り残されることのないようにすることが目指されています。\n\n出典：持続可能な開発のための2030アジェンダ（ウィキソース）'), HumanMessage(content='オープンデータの定義とは？'), AIMessage(content='オープンデータの定義は、お答えの中にある「2. オープンデータとは言えないものの、データ提供システムの維持管理に要するコストを限定された利用者からの料金徴収でまかなうケースもある」という一部の一文によると、「データ提供システムの維持管理に要するコストを限定された利用者からの料金徴収でまかなう場合は、オープンデータとは言えないが、その取扱いに準じ、二次利用可能なルールを適用することが望ましい」とされています。')]}


# Summarize Chain：文章の要約

In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

# PDFの読込
loader = PyPDFLoader("./data2/20210615_resources_data_guideline_01.pdf")
texts = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [21]:
from langchain_openai import ChatOpenAI

model_name = "gpt-3.5-turbo-0125"

# モデルの作成
model = ChatOpenAI(
    model_name=model_name,
    max_tokens=300,
    temperature=1.2)

In [22]:
from langchain.chains.summarize import load_summarize_chain

summarize_chain = load_summarize_chain(llm=model, chain_type="stuff", verbose=False)
result = summarize_chain.invoke(texts)

In [23]:
result["output_text"]

'The Open Data Basic Guidelines were established in Japan to promote the open and accessible use of public data. The guidelines focus on the significance of open data in promoting citizen participation, improving government efficiency and transparency, and fostering innovation and economic growth. Key principles include the open publication of data in machine-readable formats, the promotion of two-way data utilization, and the establishment of mechanisms for updating and maintaining public data. The document also emphasizes collaboration among government agencies, private companies, and local governments to advance open data initiatives, with a particular focus on promoting transparency, enhancing government services, and supporting economic development. The guidelines highlight the importance of reflecting user needs, establishing guidance for limiting public access to certain data, and providing support for open data efforts at the local level. The guidelines are intended to support 

In [24]:
summarize_chain = load_summarize_chain(llm=model, chain_type="map_reduce", verbose=False)
result = summarize_chain.invoke(texts)

In [25]:
result["output_text"]

"The text discusses the adoption of the Sustainable Development Goals (SDGs) by the UN in 2015 and Japan's active involvement in SDG-related efforts. The Japanese government has established organizations and guidelines to promote SDGs implementation in sectors such as food, water, infrastructure, energy, and human development. The importance of open data initiatives for transparency, collaboration, and economic growth is emphasized, with recommendations for managing closed environments for data sharing and promoting the release and utilization of open data in various sectors. Collaboration, standardization, and regular policy review are key to promoting open data initiatives across industries, with a focus on making research data supported by public funds public."

In [26]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
  input_variables=["output_text"], 
  template="{output_text}\n上記の文章を日本語訳してください")

chain = summarize_chain | prompt | model | StrOutputParser()
result = chain.invoke({"input_documents": texts})
print(result)

2015年に設立された持続可能な開発目標（SDGs）は、2030年までに特定の目標を達成するためのグローバルな目標です。日本では、SDGメディアコンパクトや中小企業の参加などを通じて、SDGsの推進を図る取り組みが行われています。日本政府は、SDGs推進本部を設置し、持続可能性の原則をビジネス運営に取り入れることを提唱しています。効率的なデータ共有を通じて社会課題の解決と経済成長を促進するために、オープンデータポリシーが推進されています。政府機関がデータの利用とアクセシビリティを確保するために従うガイドラインも設けられています。日本は、グローバルな持続可能性を目指した政府と


# Sequential Chain

### SimpleSequentialChain

In [27]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMCheckerChain, SimpleSequentialChain

llm = OpenAI(temperature=0.7)

# 1つ目のChain
template = "{subject}に関する問題を日本語で作成してください。答えは作成しないでください"
llm_chain = LLMChain(
    llm=model,
    prompt=PromptTemplate.from_template(template),
)

# 2つ目のChain
check_chain = LLMCheckerChain.from_llm(
    llm=llm)

# Chain
overall_chain = SimpleSequentialChain(
    chains=[llm_chain, check_chain],
    verbose=True)

In [28]:
# Simple Sequential Chainの実行
overall_chain.invoke("動物")



> Entering new SimpleSequentialChain chain...
1. 地球温暖化の影響で、動物たちの生息環境が変わっている。それに伴い、どのような問題が動物たちに影響を与えているか考えてみてください。
2. 野生動物の密猟や違法取引が依然として続いています。この問題にどのように取り組むべきだと考えますか？
3. 日本の国内外で行われている鳥や動物の釣り針を嘴に仕込んで不法に捕獲する方法を取り締まるためにはどうすればよいか考えてみてください。
4. 人間との共存が難しい野生動物の例と、その理由について考えてみてください。
5. 絶滅危惧種の動物の保護に関する取り組みは必要不可欠ですが、そのためにはどのような支援が必要だと


> Entering new SequentialChain chain...

> Finished chain.

1. 地球温暖化の影響で、動物たちの生息環境が変わっていることにより、多くの問題が生じています。気温の上昇により、動物たちの生活に必要な水や食料が不足することや、生息地が破壊されることで生態系が崩れることが挙げられます。また、気候変動により環境が不安定になることで、動物たちの生殖や繁殖が困難になることも問題として挙げられます。

2. 野生動物の密猟や違法取引は、野生動物の個体数を減少させ、生態系を破壊することにつながります。この問題に

> Finished chain.


{'input': '動物',
 'output': '\n1. 地球温暖化の影響で、動物たちの生息環境が変わっていることにより、多くの問題が生じています。気温の上昇により、動物たちの生活に必要な水や食料が不足することや、生息地が破壊されることで生態系が崩れることが挙げられます。また、気候変動により環境が不安定になることで、動物たちの生殖や繁殖が困難になることも問題として挙げられます。\n\n2. 野生動物の密猟や違法取引は、野生動物の個体数を減少させ、生態系を破壊することにつながります。この問題に'}

### SequentialChain

In [29]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import SequentialChain

# 1つ目のChain
template1 = "{theme}に関する{target}向けのブログ記事のタイトルを考えてください。"
llm_chain1 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template1),
    output_key="title"
)

# 2つ目のChain
template2 = "ブログ記事「{title}」の見出しを考えてください。"
llm_chain2 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template2),
    output_key="headline"
)

# Chain
overall_chain = SequentialChain(
    chains=[llm_chain1, llm_chain2],
    input_variables=["theme", "target"],
    output_variables=["title", "headline"],
    verbose=True)

In [30]:
# Chainの実行
overall_chain.invoke({
    "theme":"観光",
    "target": "日本の富裕層"})



> Entering new SequentialChain chain...

> Finished chain.


{'theme': '観光',
 'target': '日本の富裕層',
 'title': '\n\n1. "日本の美しい風景を贅沢に楽しむ旅"\n2. "プライベートジェットで巡る日本の絶景スポット"\n3. "贅沢な温泉旅館で極上の癒しを体験"\n4. "ヘリコプターで満喫する日本の絶景ツアー"\n5. "日本の伝統とラグジュアリーが融合する贅沢旅行"\n6. "最高級ホテルで味わう日本のおもてなしと美食"\n7. "日本の美術や文化に触れるVIPな観光プラン"\n8. "一流のゴルフコースで日本の自然を満喫する旅"\n9. "穴場のリゾート地で贅沢なリラックスタイムを過ごす"\n',
 'headline': '\n\n1. "日本の美しい風景を贅沢に楽しむ旅"\n2. "プライベートジェットで巡る日本の絶景スポット"\n3. "温泉と贅沢が融合する旅：ヘリコプターで満喫する日本の温泉ツアー"\n4. "日本の伝統とラグジュアリーが融合する贅沢旅行"\n5. "最高級ホテルで味わう日本のおもてなしと美食"\n6. "日本の美術や文化に触れるVIPな観光プラン"\n7. "一流のゴルフコースで日本の自然を満喫する旅"\n8. "穴場のリゾート地で贅沢なリラックスタイムを過ごす"\n9. "日本の秘境で体'}